How were places/people/events reported in Cairo? what changed? who was associated together? 
Look at cooccurence, ner, word embeddings, topic models

In [1]:
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import os
import gensim
from gensim.corpora import Dictionary
from gensim.similarities import MatrixSimilarity
from gensim.models import ldamodel, doc2vec, LsiModel 
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full
import nltk
# nltk.download('punkt')
import string
import csv
import math
import statistics
import datetime
from nltk.corpus import stopwords
from nltk.util import ngrams
# nltk.download('stopwords')
from collections import OrderedDict, Counter, namedtuple

import networkx as nx
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.layouts import row, column
import random
import codecs, difflib, Levenshtein, distance
import rpy2
from datasketch import MinHash

import importlib
ldavis = importlib.util.find_spec("pyLDAvis")
print(ldavis)
import pyLDAvis.gensim as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')
nlp = spacy.load('en_core_web_lg')

ModuleSpec(name='pyLDAvis', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1145cd4a8>, origin='/Users/zgl3n/Workspace/pyLDAvis/pyLDAvis/__init__.py', submodule_search_locations=['/Users/zgl3n/Workspace/pyLDAvis/pyLDAvis'])


In [2]:
# Read in file for issue
full_issue = pd.read_csv('ocr_test_magazine_arab_scribe_ordered.csv')


In [3]:
# If has some ordered, do the following commands
full_issue['contains_image'].fillna(value=False, inplace=True)

for index, row in full_issue.iterrows():
    if math.isnan(row['page_number']):
        pgn = row['base_file_name'].split('imagefile')[0][-3:]
        pgn = pgn.split('_')[0]
        full_issue.loc[index, 'page_number'] = int(pgn)

groupby_df = full_issue.groupby('page_number')['google_vision_text'].apply(' '.join).reset_index()
full_issue = full_issue.drop_duplicates(subset=['page_number'], keep='first')
full_issue = full_issue.drop(columns='google_vision_text')
final_df = pd.merge(full_issue, groupby_df, on='page_number', how='outer')
full_issue = final_df.drop(columns='id')

In [11]:
# Process Text Data. Process Full Issue
def custom_tokenize(text):
    if not text:
#       print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return nltk.word_tokenize(text)

def process_text(df, punc):

    final_doc = []
    for index, row in df.iterrows():
        page = []
        raw_text = row['google_vision_text']
        tokens = custom_tokenize(raw_text)
        for t in tokens:
            if punc:
                if t.lower() in string.punctuation:
                    continue
                elif t.lower() in stopwords.words('english'):
                    continue
                else:
                    final_doc.append(t.lower())
                    page.append(t.lower())
            else: 
                final_doc.append(t.lower())
    text = ' '.join(final_doc)
    return final_doc, text

order_list, order_text = process_text(full_issue, True)

scribe arab review 4th year vol vii 5 – january 1964 monthly magazine published french english german italian spanish towards arab reconciliation fifth year high dam arab observer non-aligned weekly chou en-lai u.a.r problem transportation africa monthly close-up reflection trip life encounter people many strange place along many strange road ... every- thing accomplished ought ought us charles dickens 1812-1870 arab summit 110,000 laureates honoured scribe arab review contents page editorial political scene mr. chou-en-lai 's visit cairo re-affirmation peaceful coexistance co-operation fifth year high dam course nile diverted next may ... ... education day u.a.r pays solemn tribute pioneers arab affairs towards arab reconciliation international federation petrol- eum chemical workers uar elected overwhelming majority vice-presidency meaningful presence ... documents .. economic scribe studies investigations eco- nomic social problems africa 5 problem transport africa uar banking syste

In [18]:
# Process Text Data. Process Each Page
all_pages_text = []
all_pages_token = []
for i in range(0, len(full_issue.index)):
    order_list, order_text = process_text(full_issue.iloc[[i]], True)
    all_pages_text.append(order_text)
    all_pages_token.append(order_list)
    name = 'test_texts/'+str(i)+'_text.txt'
    f = open(name, 'wt', encoding='utf-8')
    f.write(order_text)

In [30]:
from nmf import NMF
model = NMF(files='test_texts', topics=50)
print(model)
# the top terms in each topic
#  self.write_json('doc_to_topics.json', self.docs_to_topics)
#     self.write_json('topic_to_words.json', self.topics_to_words)
print(model.topics_to_words) # top terms in each topic

# # the presence of each topic in each document
print(model.docs_to_topics) # presence of each topic in each document

# # the documents by topics matrix; shape = (documents, topics)
# print(model.documents_by_topics)

# # the topics by terms matrix; shape = (topics, terms)
# print(model.topics_by_terms)

# import codecs, json
# with codecs.open('test_docs_to_topics.json', 'w', 'utf8') as f:
#      f.write(json.dumps(model.docs_to_topics, sort_keys = True, ensure_ascii=False))

defaultdict(<class 'list'>, {0: ['said', 'algeria', 'arab', 'november', 'uar', 'mr', 'refugees', 'brandt', 'agreement', 'president'], 1: ['monastery', 'century', 'st', 'catherine', 'sinai', 'moses', 'mt', 'chapel', 'stands', 'monks'], 2: ['shall', 'assembly', 'article', 'national', 'election', 'member', 'candidate', 'law', 'votes', 'constituency'], 3: ['transport', 'roads', 'africa', 'road', 'south', 'air', 'network', 'kilometres', 'traffic', 'sea'], 4: ['china', 'chinese', 'peking', 'non', 'chou', 'government', 'disarmament', 'lai', 'en', 'moscow'], 5: ['bank', 'banks', 'banking', 'million', 'commercial', 'egypt', 'national', 'law', 'branches', 'cent'], 6: ['biennale', 'prize', 'works', 'martin', 'painter', 'artistic', 'blanco', 'manuel', 'saint', 'sculptor'], 7: ['israel', 'arab', 'heads', 'conference', 'arabs', 'state', 'ranks', 'jordan', 'agents', 'hoped'], 8: ['cubic', 'dam', 'metres', 'feddans', 'high', 'work', 'nile', 'billion', 'volume', '000'], 9: ['bourguiba', 'nasser', 'binz

Co-Occurence Matrices of Ngrams

In [ ]:
def custom_tokenize(text):
    if not text:
#       print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return nltk.word_tokenize(text)

def process_text(df, types, graph_settings):
    doc = []
    final_doc = []
    for index, row in df.iterrows():
        raw_text = row['google_vision_text']
        tokens = custom_tokenize(raw_text)
        page_terms = ''
        for t in tokens:
            if t in string.punctuation:
                pass
            elif t in stopwords.words('english'):
                pass
            else:
                page_terms += t.lower() + ' '
        doc.append(page_terms)

    for sent in doc:
        sent_terms = ''
        spacy_text = nlp(sent)
        for ent in spacy_text.ents:
            if ent.label_ in types:
                sent_terms += ent.text + ' '
        final_doc.append(sent_terms)
    return create_matrix(final_doc, graph_settings)
    
def create_matrix(ents, graph_settings):
    count_model = CountVectorizer(ngram_range=(1,1)) # default unigram model
    X = count_model.fit_transform(ents)
    Xc = (X.T * X)
    Xc.setdiag(0)
    vocab = count_model.vocabulary_
    vocab2 = {y:x for x,y in vocab.items()}
    return create_network(Xc, vocab2, graph_settings)
    #ALTERNATIVE WAY TO COMPUTE MATRIX
    # occurrences = OrderedDict((name, OrderedDict((name, 0) for name in termSplit)) for name in termSplit)
    # # Find the co-occurrences:
    # for l in document:
    #     for i in range(len(l)):
    #         for item in l[:i] + l[i + 1:]:
    #             occurrences[l[i]][item] += 1
    # # Print the matrix:
    # print(' ', ' '.join(occurrences.keys()))
    # for name, values in occurrences.items():
    #     print(name, ' '.join(str(i) for i in values.values()))
    
def create_network(matrix, vocab, graph_settings):
    G = nx.from_scipy_sparse_matrix(matrix)
    H = nx.relabel_nodes(G, vocab)
    data = json_graph.node_link_data(H)
    T = json_graph.node_link_graph(data)
    ns = list(T.nodes)
    es = list(T.edges)
    final_nodes = []
    for n in G.nodes:
        nod = {'name': ns[n], 'id':n}
        final_nodes.append(nod)

    N = len(T.nodes)
    counts = np.zeros((N, N))
    for e in G.edges(data=True):
        source, target, w = e
        counts[[source], [target]] = w['weight']
        counts[[target], [source]] = w['weight']
    print(len(final_nodes))
    return draw_graph(counts, final_nodes, graph_settings, ns)
        
def draw_graph(counts, nodes, graph_settings, list_nodes):
    xname = []
    yname = []
    color = []
    alpha = []
#     colormap = ["#444444", "#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99","#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a"]
    for i, node1 in enumerate(nodes):
        for j, node2 in enumerate(nodes):
            xname.append(node1['name'])
            yname.append(node2['name'])

            alpha.append(min(counts[i,j]/4.0, 0.9) + 0.1)

    for i in range(len(xname)):
        al = alpha[i]
        if  al == 0.35:
            color.append('#ce93d8')
        elif al == 0.6:
            color.append('#ba68c8')
        elif al == 0.85:
            color.append('#9c27b0')
        elif al == 1.0:
            color.append('#7b1fa2')
        else:
            color.append('lightgrey')
    
    source = ColumnDataSource(data=dict(
        xname=xname,
        yname=yname,
        colors=color,
        alphas=alpha,
        count=counts.flatten(),
    ))

    p = figure(title=graph_settings['title'],
               x_axis_location="above", tools="hover,save",
               x_range=list(reversed(list_nodes)), y_range=list_nodes)

    p.plot_width = graph_settings['width']
    p.plot_height = graph_settings['height']
    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "5pt"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = np.pi/3

    p.rect('xname', 'yname', 0.9, 0.9, source=source,
           color='colors', alpha='alphas', line_color=None,
           hover_line_color='black', hover_color='colors')

    p.select_one(HoverTool).tooltips = [
        ('names', '@yname, @xname'),
        ('count', '@count'),
    ]

   

    return p # show the plot

NER Co-Occurence of Ngrams

Topic Models with Bootstrap Shuffle & Multiple Runs

Word Vectors with Bootstrap Shuffle